# 📝 EfficientDet Test

## requirement 수행

In [1]:
# test 관련 파일이 있는 경로로 이동
%cd automl/efficientdet

/aiffel/aiffel/EfficientDet/automl/efficientdet


In [2]:
# requirement 설치
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-66xz_n_h
  Running command git clone --filter=blob:none -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-66xz_n_h
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


## test 시 필요한 파라미터 설정

In [3]:
import hparams_config

infer_config = hparams_config.get_efficientdet_config('efficientdet-d2')

# config의 특정 항목을 update
infer_config.model_name = 'efficientdet-d2'
infer_config.model_dir = '/aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt'
# infer_config의 num_classes는 4로 바뀌어야 함. 
infer_config.num_classes = 4
infer_config.is_training_bn = False
infer_config.nms_configs.score_thresh = 0.4

print(infer_config)

act_type: swish
alpha: 0.25
anchor_scale: 4.0
apply_bn_for_resampling: true
aspect_ratios:
- 1.0
- 2.0
- 0.5
autoaugment_policy: null
backbone_config: null
backbone_name: efficientnet-b2
box_class_repeats: 3
box_loss_weight: 50.0
ckpt_var_scope: null
clip_gradients_norm: 10.0
conv_after_downsample: false
conv_bn_act_pattern: false
data_format: channels_last
dataset_type: null
delta: 0.1
drop_remainder: true
first_lr_drop_epoch: 200.0
fpn_cell_repeats: 5
fpn_config: null
fpn_name: null
fpn_num_filters: 112
fpn_weight_method: null
gamma: 1.5
grad_checkpoint: false
grid_mask: false
heads:
- object_detection
image_size: 768
img_summary_steps: null
input_rand_hflip: true
iou_loss_type: null
iou_loss_weight: 1.0
is_training_bn: false
jitter_max: 2.0
jitter_min: 0.1
label_map: null
label_smoothing: 0.1
learning_rate: 0.08
loss_scale: null
lr_decay_method: cosine
lr_warmup_epoch: 1.0
lr_warmup_init: 0.008
map_freq: 5
max_instances_per_image: 100
max_level: 7
mean_rgb:
- 123.675
- 116.28
- 103.

## 학습된 가중치를 사용하여 모델 빌딩

In [4]:
import inference
from tf2 import efficientdet_keras
import tensorflow as tf

model = efficientdet_keras.EfficientDetModel(config=infer_config)
model.build((None, None, None, 3))
print('#### checkpoint name:', tf.train.latest_checkpoint(infer_config.model_dir))
model.load_weights(tf.train.latest_checkpoint(infer_config.model_dir))
model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/tmp/__autograph_generated_filez2k3gy83.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  ag__.for_stmt(ag__.ld(self).updates, None, loop_body, get_state, set_state, (), {'iterate_names': 'u'})
/aiffel/aiffel/EfficientDet/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:


#### checkpoint name: /aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt/ckpt-100
Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b2 (Model)     multiple                  7267314   
                                                                 
 resample_p6 (ResampleFeatur  multiple                 39984     
 eMap)                                                           
                                                                 
 resample_p7 (ResampleFeatur  multiple                 0         
 eMap)                                                           
                                                                 
 fpn_cells (FPNCells)        multiple                  678479    
                                                                 
 class_net (ClassNet)        multiple                  52788     
                                                    

## Test 결과를 시각화하기 위한 준비

In [5]:
import time

class ExportModel(tf.Module):

  def __init__(self, model):
    super().__init__()
    self.model = model

  @tf.function
  def f(self, imgs):
    #model(imgs, training=False, post_mode='global')
    return self.model(imgs, training=False, post_mode='global')

export_model = ExportModel(model)

In [6]:
# label 이름 설정
labels_to_names =  {
    0 : 'Car', 
    1 : 'Truck', 
    2 : 'Bus', 
    3 : 'Pedestrian'
}

In [7]:
# test 이미지에 박스와 confidence score 출력하는 함수
def get_detected_img(export_model, img_array, is_print=True):
    # automl efficent은 반환 bbox 좌표값이 원본 이미지 좌표값으로 되어 있으므로 별도의 scaling작업 필요 없음. 
    '''
    height = img_array.shape[0]
    width = img_array.shape[1]
    '''
    # cv2의 rectangle()은 인자로 들어온 이미지 배열에 직접 사각형을 업데이트 하므로 그림 표현을 위한 별도의 이미지 배열 생성. 
    draw_img = img_array.copy()

    # bounding box의 테두리와 caption 글자색 지정
    green_color=(0, 255, 0)
    red_color=(255, 0, 0)
    blue_color=(0, 0, 255)
    pink_colo=(255, 0, 255)
    white_color=(255, 255, 255)

    # cv2로 만들어진 numpy image array를 tensor로 변환
    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.uint8)[tf.newaxis, ...]
    #img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)[tf.newaxis, ...]

    # efficientdet 모델을 다운로드 한 뒤 inference 수행. 
    start_time = time.time()
    # automl efficientdet 모델은 boxes, score, classes, num_detections를 각각 Tensor로 반환. 

    boxes, scores, classes, valid_len = export_model.f(img_tensor)
    # Tensor값을 시각화를 위해 numpy 로 변환. 
    boxes = boxes.numpy()
    scores = scores.numpy()
    classes = classes.numpy()
    valid_len = valid_len.numpy()

    # detected 된 object들을 iteration 하면서 정보 추출.
    for i in range(valid_len[0]):
        # detection score를 iteration시 마다 높은 순으로 추출하고 SCORE_THRESHOLD보다 낮으면 loop 중단. 
        score = scores[0, i]

        # detected된 object들은 scale된 기준으로 예측되었으므로 다시 원본 이미지 비율로 계산
        box = boxes[0, i]

        ''' **** 주의 ******
        box는 ymin, xmin, ymax, xmax 순서로 되어 있음. 또한 원본 좌표값으로 되어 있음. '''
        left = box[1]
        top = box[0] 
        right = box[3] 
        bottom = box[2] 

        # class id 추출하고 class 명으로 매핑
        class_id = classes[0, i]
        caption = "{}: {:.4f}".format(labels_to_names[class_id], score)
#         print(caption)
        #cv2.rectangle()은 인자로 들어온 draw_img에 사각형을 그림. 위치 인자는 반드시 정수형.
        
        if labels_to_names[class_id] == 'Car': # 객체가 car이면 박스 색깔 green
            color = green_color
        elif labels_to_names[class_id] == 'Truck': # 객체가 truck이면 박스 색깔 red
            color = red_color
        elif labels_to_names[class_id] == 'Bus': # 객체가 bus이면 박스 색깔 blue
            color = blue_color
        else:                                   # 객체가 pedestrian이면 박스 색깔 pink
            color = pink_color
            
        cv2.rectangle(draw_img, (int(left), int(top)), (int(right), int(bottom)), color=color, thickness=2)
        cv2.putText(draw_img, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, white_color, 1)

#     if is_print:
#         print('Detection 수행시간:',round(time.time() - start_time, 2),"초")

    return draw_img

## Test 할 이미지 확인

In [8]:
%cd ../../../AIFFELTHON/real_dataset/test_data

/aiffel/aiffel/AIFFELTHON/real_dataset/test_data


In [ ]:
import os

images = os.listdir('images')
images

## Test 진행

In [ ]:
import cv2
import matplotlib.pyplot as plt

for i in images[10:15]:
    img_array = cv2.cvtColor(cv2.imread('images/'+i), cv2.COLOR_BGR2RGB)

    draw_img = get_detected_img(export_model, img_array, is_print=True)
    plt.figure(figsize=(16, 16))
    plt.imshow(draw_img)